In [ ]:
# -*- coding: utf-8 -*-
"""
🔁 اتوانکودر KPI از تنها فایل 5-دقیقه‌ای — نسخهٔ نوت‌بوکی/تک‌سلولی (بدون argparse)
------------------------------------------------------------------------------
✅ فقط تغییراتِ لازم که توافق کردیم:
1) resample علّی:  label='right', closed='right'  +  shift(1)   (بدون لوک‌اِهد)
2) Warm-up درست:  پیدا کردن نقطهٔ شروع با mask.idxmax() (بدون باگ ایندکس)
بقیهٔ لوله دقیقاً مثل قبل.

ورودی: CSV با ستون‌ها: datetime, TF, mkt_adx, mkt_atr_rel, mkt_liq (فقط 5min)
خروجی‌ها: ae_encoder.pt, ae_full.pt, ae_inference_config.json, z_train.csv,
          ae_training_metrics.json, wide_sample.(parquet/csv) + AE_artifacts.zip
"""

# ==============================
# 0) ⚙️ پارامترها (قابل تغییر)
# ==============================
KPI_5M_CSV = r"C:\Users\Hesam\Desktop\Modular\AE\EURUSD_5m_kpis.csv"   # ← ورودی 5m
OUTDIR     = r"C:\Users\Hesam\Desktop\Modular\AE"                      # ← مسیر خروجی
ZIPNAME    = "AE_artifacts"                                           # ← نام ZIP (بدون پسوند)

# هایپرپارامترهای AE
K        = 8        # بُعد نهفته z
HIDDEN   = 32       # نرون‌های لایهٔ میانی
EPOCHS   = 20
BATCH    = 1024
LR       = 1e-3
ENC_BATCH= 4096     # بچ رمزگذاری z

# تنظیمات TF/KPI/تلورنس
TF_LIST   = ["5min","15min","30min","1h","4h","1d"]
KPI_LIST  = ["mkt_adx","mkt_atr_rel","mkt_liq"]

RESAMPLE_RULES = {"15min":"15min","30min":"30min","1h":"1h","4h":"4h","1d":"1D"}
# در صورت نیاز mkt_liq را "sum" کن (برای نقدشوندگی تجمیعی)
AGG_MAP = {"mkt_adx":"mean","mkt_atr_rel":"mean","mkt_liq":"mean"}

TOLERANCE = {"5min":"10min","15min":"30min","30min":"1h","1h":"2h","4h":"8h","1d":"3d"}

FEATURE_COLUMNS = [
    "mkt_adx_5min","mkt_atr_rel_5min","mkt_liq_5min",
    "mkt_adx_15min","mkt_atr_rel_15min","mkt_liq_15min",
    "mkt_adx_30min","mkt_atr_rel_30min","mkt_liq_30min",
    "mkt_adx_1h","mkt_atr_rel_1h","mkt_liq_1h",
    "mkt_adx_4h","mkt_atr_rel_4h","mkt_liq_4h",
    "mkt_adx_1d","mkt_atr_rel_1d","mkt_liq_1d",
]

# ==============================
# 1) 📚 ایمپورت‌ها + ابزارک‌های لاگ
# ==============================
import os, json, shutil
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

try:
    from IPython.display import display
except Exception:
    display = print

try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **k): return x  # اگر tqdm نبود

# تکرارپذیری
def set_seed(seed=42):
    import random
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

def log_section(title: str):
    bar = "═"*88
    print("\n" + bar); print(f"📌 {title}"); print(bar)

def log_kv(key, val):
    print(f" - {key}: {val}")

# ==============================
# 2) 📥 بارگذاری دادهٔ 5m
# ==============================
os.makedirs(OUTDIR, exist_ok=True)

log_section("گام 1/8: بارگذاری CSV ورودی 5m")
need_cols = {"datetime","TF","mkt_adx","mkt_atr_rel","mkt_liq"}
df5m = pd.read_csv(KPI_5M_CSV)
missing = need_cols - set(df5m.columns)
if missing:
    raise ValueError(f"❌ ستون‌های لازم وجود ندارد: {sorted(missing)}")

# فقط 5min
df5m = df5m[df5m["TF"].astype(str).str.lower().isin(["5min","5m","5"])].copy()
if df5m.empty:
    raise ValueError("❌ فایل باید سطرهای TF=5min داشته باشد.")

# زمان UTC و مرتب‌سازی
df5m["datetime"] = pd.to_datetime(df5m["datetime"], utc=True, errors="coerce")
df5m = df5m.dropna(subset=["datetime"]).sort_values("datetime").reset_index(drop=True)

log_kv("شمار کل رکوردهای 5m", len(df5m))
log_kv("زمان آغاز", df5m["datetime"].iloc[0])
log_kv("زمان پایان", df5m["datetime"].iloc[-1])

base = (df5m[["datetime"]+KPI_LIST]
        .drop_duplicates("datetime")
        .set_index("datetime")
        .sort_index()
        .astype("float32"))
print("✅ دادهٔ پایه آماده شد.")

# ==============================
# 3) 🧱 ساخت لانگ چند-TF با Resample (علّی)
# ==============================
log_section("گام 2/8: ساخت لانگ چند-TF با Resample از روی 5m (علّی)")

long_frames = []
tmp5 = base.copy().reset_index(); tmp5["TF"] = "5min"
long_frames.append(tmp5[["datetime","TF"] + KPI_LIST])

for tf, rule in RESAMPLE_RULES.items():
    agg_ops = {k: AGG_MAP.get(k, "mean") for k in KPI_LIST}
    # ✅ علّی: برچسب انتهای پنجره + بسته‌شدن از راست + شیفت یک‌گام (بدون لوک‌اِهد)
    res = base.resample(rule, label='right', closed='right').agg(agg_ops)
    res = res.shift(1)
    res = res.dropna(how="all").astype("float32")
    r = res.reset_index(); r["TF"] = tf
    long_frames.append(r[["datetime","TF"] + KPI_LIST])

df_long = pd.concat(long_frames, ignore_index=True).sort_values(["datetime","TF"]).reset_index(drop=True)

log_kv("شمار رکورد لانگ", len(df_long))
print("نمونه‌ای از لانگ:"); display(df_long.head(10))

# ==============================
# 4) 🔄 لانگ → واید ۱۸ستونه با merge_asof
# ==============================
log_section("گام 3/8: هم‌ترازی as-of روی محور 5m و ساخت واید ۱۸ستونه")

tf_frames = {}
for tf in TF_LIST:
    tfi = (df_long[df_long["TF"] == tf][["datetime"] + KPI_LIST]
           .drop_duplicates("datetime")
           .set_index("datetime")
           .sort_index()
           .astype("float32"))
    tf_frames[tf] = tfi

if tf_frames["5min"].empty:
    raise ValueError("❌ سری 5min خالی است.")

base_wide = tf_frames["5min"].copy().add_suffix("_5min").reset_index()

for tf in TF_LIST[1:]:
    right = tf_frames[tf].add_suffix(f"_{tf}").reset_index()
    tol = pd.Timedelta(TOLERANCE[tf])
    base_wide = pd.merge_asof(
        base_wide.sort_values("datetime"),
        right.sort_values("datetime"),
        on="datetime",
        direction="backward",
        tolerance=tol
    )

wide = base_wide.dropna().set_index("datetime")
wide = wide[FEATURE_COLUMNS].astype("float32")

log_kv("شمار ردیف‌های واید (قبل از Warm-up)", len(wide))
print("نمونه‌ای از واید:"); display(wide.head(5))

# ==============================
# 4.5) 🔥 Warm-up صحیح (بدون باگ idxmax)
# ==============================
def compute_warmup_start(wide_df, tf_list, kpi_list):
    first_valid_times = []
    for tf in tf_list:
        cols = [f"{k}_{tf}" for k in kpi_list]
        if not all(c in wide_df.columns for c in cols):
            continue
        mask_all = wide_df[cols].notna().all(axis=1)
        if mask_all.any():
            # ✅ idxmax خودش timestamp (index label) را برمی‌گرداند
            first_valid_times.append(mask_all.idxmax())
    return max(first_valid_times) if first_valid_times else wide_df.index.min()

warmup_start = compute_warmup_start(wide, TF_LIST, KPI_LIST)
before_n = len(wide)
wide = wide.loc[warmup_start:]
after_n = len(wide)
print(f"🔥 Warm-up → {before_n - after_n} ردیف ابتدایی حذف شد. شروع از: {warmup_start}")

# (اختیاری) حاشیهٔ بیشتر:
# wide = wide.loc[warmup_start + pd.Timedelta('1D'):]

# (اختیاری) تضمین بدون NaN
assert not wide.isna().any().any(), "پس از Warm-up هنوز NaN داریم؛ تولرانس/شیفت را بررسی کن."
print("✅ Warm-up سالم: هیچ NaNی باقی نمانده.")

# ==============================
# 5) 🧠 تعریف مدل AE
# ==============================
log_section("گام 4/8: تعریف مدل اتوانکودر (بدون اسکیلر)")

class AE(nn.Module):
    def __init__(self, d_in=18, d_hidden=32, k=8):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(d_in, d_hidden),
            nn.ReLU(),
            nn.Linear(d_hidden, k)
        )
        self.decoder = nn.Sequential(
            nn.Linear(k, d_hidden),
            nn.ReLU(),
            nn.Linear(d_hidden, d_in)
        )
    def forward(self, x):
        z = self.encoder(x)
        xhat = self.decoder(z)
        return xhat, z

print("✅ مدل AE آماده است.")

# ==============================
# 6) 🎯 آموزش AE با نوار پیشرفت
# ==============================
log_section("گام 5/8: آموزش AE")

X = wide.values.astype(np.float32, copy=False)
ds = TensorDataset(torch.from_numpy(X))
dl = DataLoader(ds, batch_size=BATCH, shuffle=True, drop_last=False)

model = AE(d_in=X.shape[1], d_hidden=HIDDEN, k=K).to(device)
opt = torch.optim.Adam(model.parameters(), lr=LR)
crit = nn.L1Loss()

best_loss = float("inf"); best_state = None; hist = []
model.train()
pbar = tqdm(range(1, EPOCHS+1), desc="Training", leave=True)
for ep in pbar:
    ep_loss = 0.0
    for (xb,) in dl:
        xb = xb.to(device)
        opt.zero_grad()
        xhat, _ = model(xb)
        loss = crit(xhat, xb)
        loss.backward()
        opt.step()
        ep_loss += loss.item() * xb.size(0)
    ep_loss /= len(ds)
    hist.append(float(ep_loss))
    pbar.set_postfix({"mae": f"{ep_loss:.6f}"})
    if ep_loss < best_loss:
        best_loss = ep_loss
        best_state = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}

if best_state is not None:
    model.load_state_dict(best_state)

print(f"✅ آموزش تمام شد. بهترین MAE: {best_loss:.6f}")

# ==============================
# 7) 🧪 محاسبهٔ z برای کل داده
# ==============================
log_section("گام 6/8: محاسبهٔ بردارهای نهفتهٔ z")

model.eval(); outs = []
with torch.no_grad():
    for i in tqdm(range(0, len(X), ENC_BATCH), desc="Encoding", leave=False):
        xb = torch.from_numpy(X[i:i+ENC_BATCH]).to(device)
        _, z = model(xb)
        outs.append(z.detach().cpu().numpy())

Z = np.vstack(outs)
z_cols = [f"z{i+1}" for i in range(Z.shape[1])]
df_z = pd.DataFrame(Z, index=wide.index, columns=z_cols)

print("✅ z آماده شد."); display(df_z.head())

# ==============================
# 8) 💾 ذخیرهٔ آرتیفکت‌ها + ZIP
# ==============================
log_section("گام 7/8: ذخیرهٔ آرتیفکت‌ها و ساخت ZIP")

wide_sample_path = os.path.join(OUTDIR, "wide_sample.parquet")
try:
    wide.iloc[:1000].to_parquet(wide_sample_path)
    sample_written = wide_sample_path
except Exception:
    sample_written = os.path.join(OUTDIR, "wide_sample.csv")
    wide.iloc[:1000].to_csv(sample_written, index=True)

# ذخیرهٔ مدل‌ها و متا
full_path = os.path.join(OUTDIR, "ae_full.pt")
enc_path  = os.path.join(OUTDIR, "ae_encoder.pt")
torch.save(model.state_dict(), full_path)
torch.save(model.encoder.state_dict(), enc_path)

with open(os.path.join(OUTDIR, "ae_inference_config.json"), "w", encoding="utf-8") as f:
    json.dump({"feature_columns": FEATURE_COLUMNS, "input_dim": 18}, f, indent=2, ensure_ascii=False)

df_z.to_csv(os.path.join(OUTDIR, "z_train.csv"))

with open(os.path.join(OUTDIR, "ae_training_metrics.json"), "w", encoding="utf-8") as f:
    json.dump({
        "loss_history": list(map(float, hist)),
        "k": int(K),
        "hidden": int(HIDDEN),
        "epochs": int(EPOCHS),
        "batch": int(BATCH),
        "device": device,
        "n_samples": int(len(X)),
        "feature_columns": FEATURE_COLUMNS,
        "note": "آموزش AE بدون اسکیلر از تنها ورودی 5min (resample→align + causal + warmup)",
        "agg_map": AGG_MAP,
        "input_csv": KPI_5M_CSV
    }, f, indent=2, ensure_ascii=False)

# ساخت ZIP کل پوشهٔ خروجی
zip_base = os.path.join(OUTDIR, ZIPNAME)
shutil.make_archive(zip_base, 'zip', OUTDIR)
print(f"📦 ZIP ساخته شد: {zip_base}.zip")

# ==============================
# 9) 📋 جمع‌بندی
# ==============================
log_section("گام 8/8: جمع‌بندی آرتیفکت‌ها")
summary = {
    "outdir": OUTDIR,
    "zip": os.path.join(OUTDIR, f"{ZIPNAME}.zip"),
    "files": [
        "ae_encoder.pt",
        "ae_full.pt",
        "ae_inference_config.json",
        "z_train.csv",
        "ae_training_metrics.json",
        "wide_sample.parquet or wide_sample.csv"
    ]
}
import pprint; pprint.pprint(summary)
print("\n🎉 تمام شد! وزن‌های Encoder و ترتیب ستون‌ها برای اینفرنس لایو آماده‌اند.")


c:\Users\Hesam\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



════════════════════════════════════════════════════════════════════════════════════════
📌 گام 1/8: بارگذاری CSV ورودی 5m
════════════════════════════════════════════════════════════════════════════════════════
 - شمار کل رکوردهای 5m: 1189204
 - زمان آغاز: 2009-09-25 00:00:00+00:00
 - زمان پایان: 2025-08-29 06:50:00+00:00
✅ دادهٔ پایه آماده شد.

════════════════════════════════════════════════════════════════════════════════════════
📌 گام 2/8: ساخت لانگ چند-TF با Resample از روی 5m (علّی)
════════════════════════════════════════════════════════════════════════════════════════
 - شمار رکورد لانگ: 1918523
نمونه‌ای از لانگ:


datetime     TF  mkt_adx  mkt_atr_rel  mkt_liq
0 2009-09-25 00:00:00+00:00   5min      0.0          0.0      0.0
1 2009-09-25 00:05:00+00:00   5min      0.0          0.0      0.0
2 2009-09-25 00:10:00+00:00   5min      0.0          0.0      0.0
3 2009-09-25 00:15:00+00:00  15min      0.0          0.0      0.0
4 2009-09-25 00:15:00+00:00   5min      0.0          0.0      0.0
5 2009-09-25 00:20:00+00:00   5min      0.0          0.0      0.0
6 2009-09-25 00:25:00+00:00   5min      0.0          0.0      0.0
7 2009-09-25 00:30:00+00:00  15min      0.0          0.0      0.0
8 2009-09-25 00:30:00+00:00  30min      0.0          0.0      0.0
9 2009-09-25 00:30:00+00:00   5min      0.0          0.0      0.0


════════════════════════════════════════════════════════════════════════════════════════
📌 گام 3/8: هم‌ترازی as-of روی محور 5m و ساخت واید ۱۸ستونه
════════════════════════════════════════════════════════════════════════════════════════
 - شمار ردیف‌های واید (قبل از Warm-up): 1124786
نمونه‌ای از واید:


mkt_adx_5min  mkt_atr_rel_5min  mkt_liq_5min  \
datetime                                                                  
2009-09-28 00:00:00+00:00      0.390169          0.016112      0.389418   
2009-09-28 00:05:00+00:00      0.373740          0.015543      0.373237   
2009-09-28 00:10:00+00:00      0.364523          0.015422      0.364077   
2009-09-28 00:15:00+00:00      0.368070          0.017180      0.366963   
2009-09-28 00:20:00+00:00      0.371363          0.017390      0.370167   

                           mkt_adx_15min  mkt_atr_rel_15min  mkt_liq_15min  \
datetime                                                                     
2009-09-28 00:00:00+00:00       0.407749           0.016287       0.406893   
2009-09-28 00:05:00+00:00       0.407749           0.016287       0.406893   
2009-09-28 00:10:00+00:00       0.407749           0.016287       0.406893   
2009-09-28 00:15:00+00:00       0.401428           0.015551       0.400887   
2009-09-28 00:20:00+00:00       0.401428           0.015551       0.400887   

                           mkt_adx_30min  mkt_atr_rel_30min  mkt_liq_30min  \
datetime                                                                     
2009-09-28 00:00:00+00:00       0.412181           0.017203        0.41093   
2009-09-28 00:05:00+00:00       0.412181           0.017203        0.41093   
2009-09-28 00:10:00+00:00       0.412181           0.017203        0.41093   
2009-09-28 00:15:00+00:00       0.412181           0.017203        0.41093   
2009-09-28 00:20:00+00:00       0.412181           0.017203        0.41093   

                           mkt_adx_1h  mkt_atr_rel_1h  mkt_liq_1h  mkt_adx_4h  \
datetime                                                                        
2009-09-28 00:00:00+00:00     0.44417        0.022284    0.440457    0.235656   
2009-09-28 00:05:00+00:00     0.44417        0.022284    0.440457    0.235656   
2009-09-28 00:10:00+00:00     0.44417        0.022284    0.440457    0.235656   
2009-09-28 00:15:00+00:00     0.44417        0.022284    0.440457    0.235656   
2009-09-28 00:20:00+00:00     0.44417        0.022284    0.440457    0.235656   

                           mkt_atr_rel_4h  mkt_liq_4h  mkt_adx_1d  \
datetime                                                            
2009-09-28 00:00:00+00:00        0.020054    0.234381    0.184714   
2009-09-28 00:05:00+00:00        0.020054    0.234381    0.184714   
2009-09-28 00:10:00+00:00        0.020054    0.234381    0.184714   
2009-09-28 00:15:00+00:00        0.020054    0.234381    0.184714   
2009-09-28 00:20:00+00:00        0.020054    0.234381    0.184714   

                           mkt_atr_rel_1d  mkt_liq_1d  
datetime                                               
2009-09-28 00:00:00+00:00        0.019348    0.183071  
2009-09-28 00:05:00+00:00        0.019348    0.183071  
2009-09-28 00:10:00+00:00        0.019348    0.183071  
2009-09-28 00:15:00+00:00        0.019348    0.183071  
2009-09-28 00:20:00+00:00        0.019348    0.183071

...

Training: 100%|██████████| 20/20 [02:43<00:00,  8.18s/it, mae=0.001408]


✅ آموزش تمام شد. بهترین MAE: 0.001408

════════════════════════════════════════════════════════════════════════════════════════
📌 گام 6/8: محاسبهٔ بردارهای نهفتهٔ z
════════════════════════════════════════════════════════════════════════════════════════


✅ z آماده شد.


z1        z2        z3        z4        z5  \
datetime                                                                      
2009-09-28 00:00:00+00:00 -0.646668 -0.179071 -0.197576 -0.352637 -0.353497   
2009-09-28 00:05:00+00:00 -0.630080 -0.173472 -0.193809 -0.343213 -0.365045   
2009-09-28 00:10:00+00:00 -0.620685 -0.170286 -0.191751 -0.337925 -0.371561   
2009-09-28 00:15:00+00:00 -0.620216 -0.170602 -0.189776 -0.339202 -0.363339   
2009-09-28 00:20:00+00:00 -0.623499 -0.171703 -0.190558 -0.341091 -0.361043   

                                 z6        z7        z8  
datetime                                                 
2009-09-28 00:00:00+00:00 -0.443060  0.058083  0.086477  
2009-09-28 00:05:00+00:00 -0.444768  0.052885  0.092115  
2009-09-28 00:10:00+00:00 -0.445710  0.049985  0.095291  
2009-09-28 00:15:00+00:00 -0.443795  0.050102  0.097669  
2009-09-28 00:20:00+00:00 -0.443444  0.051152  0.096546


════════════════════════════════════════════════════════════════════════════════════════
📌 گام 7/8: ذخیرهٔ آرتیفکت‌ها و ساخت ZIP
════════════════════════════════════════════════════════════════════════════════════════
